# Setup

In [1]:
# !pip install datasets transformers[torch] scikit-learn scipy torchcodec

In [2]:
# !apt-get update && apt-get install -y ffmpeg

In [13]:
# !pip install --upgrade transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [1]:
import numpy as np
import datasets
import librosa
from datasets import load_dataset
from datasets import Dataset
from tqdm.auto import tqdm

from transformers import (
    AutoFeatureExtractor,
    AutoModelForAudioClassification,
    Trainer,
    TrainingArguments
)

from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr
from torch.nn import MSELoss

In [4]:
#from huggingface_hub import login

#login()

In [2]:
# Load the dataset (we'll only use 'train' for now and split it ourselves)
# Note: 'streaming=True' can be used if you're low on disk space
#dataset = load_dataset("ASLP-lab/SongEval", split='train')
#print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'TypeError: NetworkError when attempting to fetch resource.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Resolving data files:   0%|          | 0/2405 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'gender', 'annotation'],
    num_rows: 2399
})


In [2]:
 from google.colab import drive

# 1. Mount your Google Drive
 drive.mount('/content/drive')

# 2. --- !! IMPORTANT !! ---
#    Update this path to point to your dataset folder on Google Drive
#    (It's the folder that contains 'dataset_infos.json', 'data', etc.)
dataset_path = "/content/drive/MyDrive/ASLP-SongEval"

# 3. Load the dataset from your local path
dataset = load_dataset(dataset_path, split='train', streaming='True')

# 4. Apply the 'decode=False' fix. This is STILL necessary.
dataset = dataset.cast_column("audio", datasets.Audio(decode=False))

print(f"--- Successfully loaded dataset from {dataset_path} ---")
print(dataset)

Mounted at /content/drive


Resolving data files:   0%|          | 0/2405 [00:00<?, ?it/s]

--- Successfully loaded dataset from /content/drive/MyDrive/ASLP-SongEval ---
IterableDataset({
    features: ['audio', 'gender', 'annotation'],
    num_shards: 1
})


In [3]:
feature_extractor = AutoFeatureExtractor.from_pretrained(
    "MIT/ast-finetuned-audioset-10-10-0.4593"
)

preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [4]:

# REPLACE your old function with this new one
def ast_preprocess_from_path(example):

    audio_path = example["audio"]["path"]

    target_sr = 16000 # AST's expected sample rate
    target_sec = 10

    try:
        # 1. Get total duration (cheaply)
        duration = librosa.get_duration(path=audio_path)

        # 2. Pick a random start time (in seconds)
        start_time = 0.0
        if duration > target_sec:
            start_time = np.random.uniform(0.0, duration - target_sec)

        # 3. Load ONLY the 10-second chunk from disk
        chunk, sr = librosa.load(
            audio_path,
            sr=target_sr,       # Resample to 16kHz
            offset=start_time,  # Start at our random time
            duration=target_sec # Load only 10 seconds
        )

        # 4. Pad if the chunk is < 10s (e.g., end of song or short file)
        max_length_samples = target_sr * target_sec
        if len(chunk) < max_length_samples:
            chunk = np.pad(chunk, (0, max_length_samples - len(chunk)), 'constant')

        # 5. Process the 10-second chunk
        inputs = feature_extractor(
            chunk,
            sampling_rate=target_sr,
            padding="max_length",
            return_tensors="pt"
        )

        inputs['input_values'] = inputs['input_values'].squeeze(0)

        # --- THIS IS THE FIX ---
        # Detach the tensor, move it to CPU, and convert to a simple NumPy array.
        # This uses far less RAM.
        final_input_values = inputs['input_values'].detach().cpu().numpy()

        final_labels = [
            example['Coherence_mean'],
            example['Memorability_mean'],
            example['Naturalness_mean'],
            example['Clarity_mean'],
            example['Musicality_mean']
        ]

        # Return a NEW, clean dict with just numpy arrays and lists.
        return {
            'input_values': final_input_values,
            'labels': final_labels
        }

    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

In [5]:
# 1. Take 20 examples from the stream
small_stream = dataset.take(500)

# 2. Create empty lists to store our data
all_input_values = []
all_labels = []

target_sr = 16000
target_sec = 10
max_length_samples = target_sr * target_sec

print(f"--- Starting MANUAL processing loop for 20 examples ---")

# 3. Manually loop through the stream (this is the fix)
for example in tqdm(small_stream, total=500):
    try:
        audio_path = example["audio"]["path"]

        # --- Manually load audio chunk ---
        duration = librosa.get_duration(path=audio_path)
        start_time = 0.0
        if duration > target_sec:
            start_time = np.random.uniform(0.0, duration - target_sec)

        chunk, sr = librosa.load(
            audio_path,
            sr=target_sr,
            offset=start_time,
            duration=target_sec
        )

        if len(chunk) < max_length_samples:
            chunk = np.pad(chunk, (0, max_length_samples - len(chunk)), 'constant')

        # --- Manually process with feature_extractor ---
        inputs = feature_extractor(
            chunk,
            sampling_rate=target_sr,
            padding="max_length",
            return_tensors="pt"
        )

        # --- THIS IS THE NEW, CORRECTED LABEL LOGIC ---

        # 1. Get the list of annotator dicts
        annotations = example['annotation']

        # 2. Create lists to hold all scores
        coherence_scores = []
        memorability_scores = []
        naturalness_scores = []
        clarity_scores = []
        musicality_scores = []

        # 3. Loop through each annotator
        for ann in annotations:
            coherence_scores.append(ann['Coherence'])
            memorability_scores.append(ann['Memorability'])
            naturalness_scores.append(ann['Naturalness'])
            clarity_scores.append(ann['Clarity'])
            musicality_scores.append(ann['Musicality'])

        # 4. Calculate the mean for each score
        labels = [
            np.mean(coherence_scores),
            np.mean(memorability_scores),
            np.mean(naturalness_scores),
            np.mean(clarity_scores),
            np.mean(musicality_scores)
        ]
        # --- END OF NEW LABEL LOGIC ---

        # --- Manually append plain numpy/list data ---
        all_input_values.append(inputs['input_values'].squeeze(0).detach().cpu().numpy())
        all_labels.append(labels)

    except Exception as e:
        print(f"Error processing {example.get('audio', {}).get('path', 'unknown')}: {e}")

print(f"\n--- Successfully processed {len(all_input_values)} / 20 examples ---")

# 4. Create the in-memory Dataset from our clean lists
if len(all_input_values) > 0:
    processed_dataset = Dataset.from_dict({
        'input_values': all_input_values,
        'labels': all_labels
    })

    # 5. Set format for the trainer
    processed_dataset.set_format("torch")

    # 6. Split the data
    processed_dataset = processed_dataset.train_test_split(test_size=0.1, seed=42)

    print("\n--- SUCCESSFULLY CREATED, FORMATTED, AND SPLIT DATASET ---")
    print(processed_dataset)
else:
    print("\n--- No data was processed. ---")

--- Starting MANUAL processing loop for 20 examples ---


  0%|          | 0/500 [00:00<?, ?it/s]


--- Successfully processed 500 / 20 examples ---

--- SUCCESSFULLY CREATED, FORMATTED, AND SPLIT DATASET ---
DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 50
    })
})


In [6]:
model = AutoModelForAudioClassification.from_pretrained(
    "MIT/ast-finetuned-audioset-10-10-0.4593",
    num_labels=5,                 # We are predicting 5 scores
    ignore_mismatched_sizes=True  # This is the key fix!
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Block 8: Custom Regression Trainer (FIXED for ancient transformers)

class RegressionTrainer(Trainer):
    # --- THIS IS THE FIX ---
    # We added 'num_items_in_batch' to the arguments, so the Trainer
    # can pass it in without crashing. We don't have to use it.
    def compute_loss(self, model, inputs, num_items_in_batch, return_outputs=False):
        # -----------------------

        # Get labels from inputs
        labels = inputs.pop("labels")

        # Get model outputs
        outputs = model(**inputs)
        logits = outputs.logits

        # Define the loss function
        loss_fct = MSELoss()

        # Ensure labels are float to match logits type (which is float)
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Calculate MAE
    mae = mean_absolute_error(labels, predictions)

    # Calculate Spearman Correlation (SRCC) for each of the 5 tasks
    srcc_scores = []
    for i in range(labels.shape[1]): # Iterate over each of the 5 dimensions
        # spearmanr returns a tuple (correlation, pvalue)
        correlation, p_value = spearmanr(labels[:, i], predictions[:, i])
        srcc_scores.append(correlation)

    # Calculate the average SRCC across all 5 dimensions
    avg_srcc = np.mean(srcc_scores)

    return {
        "mean_absolute_error": mae,
        "average_spearman_correlation": avg_srcc,
        "srcc_coherence": srcc_scores[0],
        "srcc_memorability": srcc_scores[1],
        "srcc_naturalness": srcc_scores[2],
        "srcc_clarity": srcc_scores[3],
        "srcc_musicality": srcc_scores[4]
    }

In [9]:
training_args = TrainingArguments(
    output_dir="./SongEval-AST-Baseline",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    logging_steps=5,  # Log more often
)

print("--- Using BULLETPROOF TrainingArguments. This will work. ---")

--- Using BULLETPROOF TrainingArguments. This will work. ---


In [10]:
# NOTE: We are using RegressionTrainer, not the default Trainer
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    # Use the new train/test splits
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

# Start the training
trainer.train()

/tmp/ipython-input-1551410090.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `RegressionTrainer.__init__`. Use `processing_class` instead.
  trainer = RegressionTrainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ahsanadil (ahsanadil-nust) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
5,44.141200
10,15.622600
15,6.117800
20,3.250800
25,3.986900
30,2.328700
35,2.980600
40,2.081900
45,2.900600
50,2.420400


TrainOutput(global_step=171, training_loss=3.4151340286895544, metrics={'train_runtime': 419.928, 'train_samples_per_second': 3.215, 'train_steps_per_second': 0.407, 'total_flos': 9.1509283160064e+16, 'train_loss': 3.4151340286895544, 'epoch': 3.0})

In [11]:
print("--- Final Evaluation on Test Set ---")
eval_results = trainer.evaluate()
print(eval_results)

--- Final Evaluation on Test Set ---


{'eval_loss': 0.38545599579811096, 'eval_mean_absolute_error': 0.5052167177200317, 'eval_average_spearman_correlation': 0.8276498622704495, 'eval_srcc_coherence': 0.8548763361668593, 'eval_srcc_memorability': 0.847779059331255, 'eval_srcc_naturalness': 0.7800930678369487, 'eval_srcc_clarity': 0.8520005876564729, 'eval_srcc_musicality': 0.8035002603607111, 'eval_runtime': 4.7277, 'eval_samples_per_second': 10.576, 'eval_steps_per_second': 5.288, 'epoch': 3.0}
